[pydantic types](https://docs.pydantic.dev/latest/concepts/types/)

In [ ]:
from fastapi import FastAPI

app = FastAPI()

## Параметры валидации
- `min_length`
- `max_length`
- `regex`
- `gt`: больше (`g`reater `t`han)
- `ge`: больше или равно (`g`reater than or `e`qual)
- `lt`: меньше (`l`ess `t`han)
- `le`: меньше или равно (`l`ess than or `e`qual)
- `alias`
- `title`
- `description`
- `deprecated`
- `include_in_schema`

## Path()
Используется для извлечения и валидации переменных из пути URL. Например, в `/items/{item_id}` 

`item_id` может быть объявлено и валидировано с использованием `Path()`.

In [ ]:
from fastapi import Path


@app.get('/items/{item_id}')
async def read_item(item_id: int = Path(..., title='The ID of the item to get')):
    return {'item_id': item_id}

## Query()
Используется для извлечения и валидации параметров строки запроса.\
Например, в `/items?page=2` параметр `page` может быть объявлен с использованием `Query()`.

In [ ]:
from fastapi import Query


@app.get('/items/')
async def read_items(q: str = Query(None, title='Query string')):
    return {'q': q}

## Header()
Используется для извлечения и валидации значений HTTP-заголовков.

In [ ]:
from fastapi import Header


@app.get('/items/')
async def read_items(user_agent: str = Header(None)):
    return {'User-Agent': user_agent}

## Cookie()
Используется для извлечения и валидации значений куки из запроса

In [ ]:
from fastapi import Cookie


@app.get('/items/')
async def read_items(ads_id: str = Cookie(None)):
    return {'ads_id': ads_id}

## Body()
Используется для извлечения и валидации данных, отправленных в теле запроса (например, в формате JSON).

In [ ]:
from fastapi import Body

from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: str = None


@app.post('/items/')
async def create_item(item: Item = Body(...)):
    return {'name': item.name, 'description': item.description}

## Form()
Используется для извлечения и валидации данных из HTML-форм, отправленных через запросы `application/x-www-form-urlencoded`.

In [ ]:
from fastapi import Form


@app.post('/login/')
async def login(username: str = Form(...), password: str = Form(...)):
    return {'username': username}

## File()
Используется для извлечения и валидации файлов, отправленных через многокомпонентные формы (`multipart/form-data`).

In [ ]:
from fastapi import File, UploadFile


@app.post('/uploadfile/')
async def upload_file(file: UploadFile = File(...)):
    return {'filename': file.filename}

## Field
Используется для детальной валидации отдельных полей модели Pydantic.\
Позволяет задать дополнительные параметры валидации для каждого поля.

In [ ]:
from pydantic import BaseModel, Field


class Item(BaseModel):
    name: str = Field(..., min_length=1, max_length=300)
    description: str = Field(None, title='The description of the item', max_length=500)
    price: float = Field(..., gt=0, description='The price must be greater than zero')
    tax: float = None


@app.post('/items/')
async def create_item(item: Item):
    return item

## Depends()
Используется для объявления зависимостей.\
Может использоваться для создания сложных зависимостей и повторного использования логики в разных частях приложения.

In [ ]:
from fastapi import Depends


def common_parameters(q: str = None, skip: int = 0, limit: int = 100):
    return {'q': q, 'skip': skip, 'limit': limit}


@app.get('/items/')
async def read_items(commons: dict = Depends(common_parameters)):
    return commons

## WebSocket
Используется для работы с WebSocket соединениями.

In [ ]:
from fastapi import WebSocket


@app.websocket('/ws')
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        data = await websocket.receive_text()
        await websocket.send_text(f'Message text was: {data}')

## Security()
Используется для добавления безопасности к API, часто в сочетании с системами аутентификации и авторизации.

In [ ]:
from fastapi import Depends, FastAPI, Security
from fastapi.security.api_key import APIKeyHeader
from fastapi.exceptions import HTTPException

api_key_header = APIKeyHeader(name='X-API-Key')


async def get_api_key(api_key_header: str = Security(api_key_header)):
    if api_key_header != 'secret_key':
        raise HTTPException(status_code=403, detail='Could not validate credentials')
    return api_key_header


@app.get('/secure-data/')
async def secure_data(api_key: str = Depends(get_api_key)):
    return {'message': 'Secure data accessed'}

### Annotated
Позволяет комбинировать типы данных с дополнительной информацией для параметров функций

In [ ]:
from typing import Annotated

In [ ]:
username: Annotated[str, Field(min_length=3, max_length=50)]

In [ ]:
email: Annotated[
    str,
    Query(
        default=None,
        example='user@example.com',
        description="The user's email address"
    ),
]

In [ ]:
user_id: Annotated[int, Path(description='The ID of the user you want to retrieve')]

In [ ]:
@app.get('/')
def get_home(q: Annotated[str | None, Query(max_length=50)] = None):
    return True